<a href="https://colab.research.google.com/github/Varchan/TUBES-CLOUD-COMPUTING-Sentimen-Analisis-dan-Visualisasi-Data-OpenBO-via-Twitter/blob/main/Tubes_Sentimen_Analisis_data_OpenBO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Tubes Cloud Computing

/content/drive/MyDrive/Tubes Cloud Computing


In [4]:
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.5 MB/s eta 0:00:00


In [3]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
df = pd.read_excel('/content/drive/MyDrive/Tubes Cloud Computing/tweet-sentiment-03.xlsx')
df['sentiment'] = df['sentiment'].replace(['Neutral'], 0)
df['sentiment'] = df['sentiment'].replace(['Positif'], 1)
df['sentiment'] = df['sentiment'].replace(['Negatif'], 2)


df


,Unnamed: 0,username,tweets,sentiment
0,0,18 December 2022 (@TheGOAT_campeon),"crot what!? <a href=""https://t.co/3W1D5suX3X""",0
1,1,å…‰ (@weiguang91),Kamu udah crot perdana blm? Wkwkw,0
2,2,Ah Aah (@Ahenakah),Yg crot begini nih bini gw yg suka,0
3,3,Adinda Sexsi (@m503oOohmd),Semalem kerasa crot punyaku sampe becek yuk re...,1
4,4,Crot2b (@crot2b),yo,0
...,...,...,...,...
3254,3254,niko nurcahyo (@nurcahyo_n72231),Harusnya..... Kak mau ngewe gak?,0
3255,3255,sara (@daywithout3),gua dah jomblo otw 3 tahun dan udeh disuruh &q...,0
3256,3256,Dewi Syalala (@PutriSyalala8),"ya jelas parah, lha setiap tahun kelahiran ana...",0
3257,3257,Jimmy P (@pinarta),Abis ngewe brutal ama 2 cewe. Masih cape. Ni u...,2


In [6]:

#import stopword
from nltk.tokenize import sent_tokenize, word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [7]:
def pre_process(text):
    # Case Folding: Lowercase
    # Merubah format teks menjadi format huruf kecil semua (lowercase).
    text = text.lower()

    # Case Folding: Removing Number
    # Menghapus karakter angka.
    text = re.sub(r"\d+", "", text)

    # Case Folding: Removing Punctuation
    # Menghapus karakter tanda baca.
    text = text.translate(str.maketrans("","",string.punctuation))

    #Case Folding: Removing Whitespace
    #Menghapus karakter kosong.
    text = text.strip()


    #Separating Sentences with Split () Method
    #Fungsi split() memisahkan string ke dalam list dengan spasi sebagai pemisah jika tidak ditentukan pemisahnya.
    pisah = text.split()

    #Filtering using sastrawi
    factory = StopWordRemoverFactory()
    stopword = factory.create_stop_word_remover()
    text =  stopword.remove(text)

    return text

df['tweets'] = df['tweets'].apply(lambda x:pre_process(x))
df

,Unnamed: 0,username,tweets,sentiment
0,0,18 December 2022 (@TheGOAT_campeon),crot what a hrefhttpstcowdsuxx,0
1,1,å…‰ (@weiguang91),kamu udah crot perdana blm wkwkw,0
2,2,Ah Aah (@Ahenakah),yg crot begini nih bini gw yg suka,0
3,3,Adinda Sexsi (@m503oOohmd),semalem kerasa crot punyaku sampe becek yuk re...,1
4,4,Crot2b (@crot2b),yo,0
...,...,...,...,...
3254,3254,niko nurcahyo (@nurcahyo_n72231),harusnya kak mau ngewe gak,0
3255,3255,sara (@daywithout3),gua dah jomblo otw tahun udeh disuruh quotpun...,0
3256,3256,Dewi Syalala (@PutriSyalala8),jelas parah lha tahun kelahiran anak rohingya ...,0
3257,3257,Jimmy P (@pinarta),abis ngewe brutal ama cewe cape ni udah mulai...,2


In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix

train_df, test_df = train_test_split(df, test_size = 0.2, random_state = 42)
print("Training data size : ", train_df.shape)
print("Test data size : ", test_df.shape)

Training data size :  (2607, 4)
Test data size :  (652, 4)


In [10]:

top_words = 10000
tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(train_df['tweets'])
list_tokenized_train = tokenizer.texts_to_sequences(train_df['tweets'])

max_review_length = 200
X_train = pad_sequences(list_tokenized_train, maxlen=max_review_length)
y_train = train_df['sentiment']


In [11]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 32)           320032    
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 373333 (1.42 MB)
Trainable params: 373333 (1.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
model.fit(X_train,y_train, epochs=20, batch_size=64, validation_split=0.2)


Epoch 1/20
33/33 [==============================] - 13s 288ms/step - loss: 0.5805 - accuracy: 0.7650 - val_loss: 0.4998 - val_accuracy: 0.7931
Epoch 2/20
33/33 [==============================] - 10s 309ms/step - loss: 0.4044 - accuracy: 0.8201 - val_loss: 0.3536 - val_accuracy: 0.8333
Epoch 3/20
33/33 [==============================] - 8s 250ms/step - loss: 0.1670 - accuracy: 0.8657 - val_loss: 0.2196 - val_accuracy: 0.8410
Epoch 4/20
33/33 [==============================] - 10s 312ms/step - loss: -0.0491 - accuracy: 0.9070 - val_loss: 0.0971 - val_accuracy: 0.8487
Epoch 5/20
33/33 [==============================] - 9s 277ms/step - loss: -0.3834 - accuracy: 0.9261 - val_loss: 0.0400 - val_accuracy: 0.8525
Epoch 6/20
33/33 [==============================] - 9s 278ms/step - loss: -0.7236 - accuracy: 0.9420 - val_loss: 0.7929 - val_accuracy: 0.7510
Epoch 7/20
33/33 [==============================] - 11s 347ms/step - loss: -1.0125 - accuracy: 0.9453 - val_loss: -0.0619 - val_accuracy: 0.85

In [13]:
list_tokenized_test = tokenizer.texts_to_sequences(test_df['tweets'])
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
y_test = test_df['sentiment']
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
#print('F1-score: ', f1_score(y_pred, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,y_pred)

21/21 [==============================] - 2s 77ms/step
Accuracy of the model :  0.8941717791411042
Confusion matrix:


array([[476,  27,   0],
       [ 18, 107,   0],
       [ 12,  12,   0]])